<a href="https://www.kaggle.com/code/quangnhatbui/finding-the-best-italian-location-in-hcm-city?scriptVersionId=133313474" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Business

We will find the best location to open a Italian restaurant in Vietnam.

## Data

- We will base on the popularity of coffee shop to define if we should open it in that neighborhood
- Position that is close to the center of the city which is district 1

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import folium
import requests

In [2]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [3]:
district = ["District 1", "District 2", "District 3","District 4"
            ,"District 7","Binh Thanh","Tan Binh","Binh Tan","Phu Nhuan", "Thu Duc"]
postal = [710000,711000,724000,728000,729000
          ,723000,721000,762000,725600,721400]
lat = [10.779320, 10.787182, 10.784265, 10.757808, 10.734065, 10.810574,10.801498, 
       10.765238, 10.799217,10.849422]
long = [106.700059, 106.749557, 106.684386, 106.701283, 106.721623, 106.709134,106.652643,
        106.603853, 106.680260,106.753664]
df = pd.DataFrame({"Neighborhood":district, "Postal code": postal, "Latitude": lat, "Longitude":long})
df

,Neighborhood,Postal code,Latitude,Longitude
0,District 1,710000,10.779320,106.700059
1,District 2,711000,10.787182,106.749557
2,District 3,724000,10.784265,106.684386
3,District 4,728000,10.757808,106.701283
4,District 7,729000,10.734065,106.721623
5,Binh Thanh,723000,10.810574,106.709134
6,Tan Binh,721000,10.801498,106.652643
7,Binh Tan,762000,10.765238,106.603853
8,Phu Nhuan,725600,10.799217,106.680260
9,Thu Duc,721400,10.849422,106.753664


In [4]:
lat = 10.762622
long = 106.660172
map_hcm = folium.Map(location=[lat,long], zoom_start=11)
for lat, lng, label in zip(df["Latitude"], df["Longitude"], df["Neighborhood"]):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color="blue",
        fill=True,
        fill_color='#3186cc',
        opacity=0.7,
        parse_html=False).add_to(map_hcm)
map_hcm

In [5]:
import json
from pandas.io.json import json_normalize

CLIENT_ID = '0E2W0K50NHZMGMNW3LP2NDJNZWBAMDHGGPTZA40IP0KGWAAJ' 
CLIENT_SECRET = 'NJSPIXE2GDYUT4YPSMXDF1HTWPR54YYYGG4U5HSTIDVN025D'
VERSION = '20180605'
district_lat = df.loc[0,"Latitude"]
district_lng = df.loc[0,"Longitude"]


In [6]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

df_1 = getNearbyVenues(df["Neighborhood"], df["Latitude"], df["Longitude"])
df_1

District 1
District 2
District 3
District 4
District 7
Binh Thanh
Tan Binh
Binh Tan
Phu Nhuan
Thu Duc


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,District 1,10.779320,106.700059,Club Lounge @ InterContinental Saigon,10.780975,106.700968,Hotel Bar
1,District 1,10.779320,106.700059,Cộng Cà Phê,10.777917,106.701266,Café
2,District 1,10.779320,106.700059,Đường Sách Nguyễn Văn Bình,10.780249,106.699404,Bookstore
3,District 1,10.779320,106.700059,Pacey Cupcakes,10.780202,106.700773,Cupcake Shop
4,District 1,10.779320,106.700059,InterContinental Saigon,10.781261,106.700786,Hotel
...,...,...,...,...,...,...,...
204,Phu Nhuan,10.799217,106.680260,Rock Cafe K13,10.802953,106.681831,Café
205,Thu Duc,10.849422,106.753664,Chợ Thủ Đức A,10.850634,106.754400,Market
206,Thu Duc,10.849422,106.753664,Dominos Pizza Kha Van can,10.846685,106.751388,Pizza Place
207,Thu Duc,10.849422,106.753664,Hủ Tiếu Bò Viên Xuân Ký,10.851161,106.754542,Noodle House


In [7]:
df_1["Venue Category"] = df_1["Venue Category"].replace("Café", "Coffee Shop")

In [8]:
hotspot = pd.get_dummies(df_1["Venue Category"], prefix="", prefix_sep="")
hotspot["Neighborhood"] = df_1["Neighborhood"]
hotspot_grouped = hotspot.groupby("Neighborhood").mean().reset_index()
hotspot_grouped

,Neighborhood,Argentinian Restaurant,Asian Restaurant,BBQ Joint,Bar,Bed & Breakfast,Beer Garden,Bookstore,Boutique,Breakfast Spot,...,Snack Place,Spa,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar
0,Binh Tan,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Binh Thanh,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.222222,0.000000
2,District 1,0.011111,0.022222,0.011111,0.022222,0.000000,0.011111,0.011111,0.011111,0.000000,...,0.000000,0.000000,0.000000,0.022222,0.011111,0.011111,0.011111,0.011111,0.077778,0.011111
3,District 2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000
4,District 3,0.000000,0.052632,0.026316,0.026316,0.000000,0.000000,0.000000,0.000000,0.026316,...,0.000000,0.026316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.315789,0.000000
5,District 4,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.095238,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.095238,0.000000
6,District 7,0.000000,0.000000,0.076923,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.230769,0.000000
7,Phu Nhuan,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000
8,Tan Binh,0.000000,0.066667,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,...,0.000000,0.000000,0.066667,0.066667,0.000000,0.000000,0.000000,0.000000,0.133333,0.000000
9,Thu Duc,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [9]:
cols = ["Neighborhood"]
indicators = ["st", "nd", "rd"]

for x in range(0, 15):
    if x < 3:
        cols.append("The {}{} famous".format(x+1, indicators[x]))
    else:
        cols.append("The {}th famous".format(x+1))

hotspot_rank = pd.DataFrame(columns=cols)
hotspot_rank["Neighborhood"] = hotspot_grouped["Neighborhood"]
for x in range(len(hotspot_grouped)):
    hotspot_rank.iloc[x, 1:] = hotspot_grouped.iloc[x,1:].sort_values(ascending=False).iloc[0:15].index
hotspot_rank

,Neighborhood,The 1st famous,The 2nd famous,The 3rd famous,The 4th famous,The 5th famous,The 6th famous,The 7th famous,The 8th famous,The 9th famous,The 10th famous,The 11th famous,The 12th famous,The 13th famous,The 14th famous,The 15th famous
0,Binh Tan,Flea Market,Hotel,Dance Studio,Argentinian Restaurant,Multiplex,Opera House,North Indian Restaurant,Noodle House,Museum,Mexican Restaurant,Middle Eastern Restaurant,Massage Studio,Market,Lounge,Korean Restaurant
1,Binh Thanh,Vietnamese Restaurant,Coffee Shop,IT Services,Colombian Restaurant,Bookstore,Food Service,Food Truck,Museum,Opera House,North Indian Restaurant,Noodle House,Multiplex,Pizza Place,Middle Eastern Restaurant,Mexican Restaurant
2,District 1,Coffee Shop,Vietnamese Restaurant,Hotel,Massage Studio,French Restaurant,Italian Restaurant,Hotel Bar,Brewery,Hotpot Restaurant,Asian Restaurant,Bar,Japanese Restaurant,Fast Food Restaurant,Sushi Restaurant,Tea Room
3,District 2,Coffee Shop,Vietnamese Restaurant,Restaurant,Argentinian Restaurant,Middle Eastern Restaurant,North Indian Restaurant,Noodle House,Museum,Multiplex,Mexican Restaurant,Park,Massage Studio,Market,Lounge,Korean Restaurant
4,District 3,Vietnamese Restaurant,Coffee Shop,French Restaurant,Seafood Restaurant,Asian Restaurant,BBQ Joint,Bar,Lounge,Breakfast Spot,Noodle House,Rock Club,Juice Bar,Spa,Cupcake Shop,Argentinian Restaurant
5,District 4,Seafood Restaurant,Snack Place,Vietnamese Restaurant,Coffee Shop,Brewery,Pizza Place,Grocery Store,Gym / Fitness Center,Mexican Restaurant,Japanese Restaurant,Asian Restaurant,Korean Restaurant,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Massage Studio
6,District 7,Vietnamese Restaurant,Flea Market,Coffee Shop,BBQ Joint,Bed & Breakfast,Residential Building (Apartment / Condo),Scandinavian Restaurant,Sushi Restaurant,Gym / Fitness Center,Argentinian Restaurant,Museum,North Indian Restaurant,Noodle House,Mexican Restaurant,Multiplex
7,Phu Nhuan,Coffee Shop,Vietnamese Restaurant,Gym / Fitness Center,BBQ Joint,Bookstore,Noodle House,Pizza Place,Multiplex,Opera House,North Indian Restaurant,Museum,Argentinian Restaurant,Middle Eastern Restaurant,Mexican Restaurant,Massage Studio
8,Tan Binh,Vietnamese Restaurant,Coffee Shop,Supermarket,Seafood Restaurant,BBQ Joint,Asian Restaurant,Breakfast Spot,Sushi Restaurant,Bubble Tea Shop,Flower Shop,Multiplex,Gym / Fitness Center,Shopping Plaza,North Indian Restaurant,Noodle House
9,Thu Duc,Pizza Place,Market,Noodle House,Argentinian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Lounge,Massage Studio,Mexican Restaurant,Middle Eastern Restaurant,Multiplex,Museum,North Indian Restaurant,Opera House


In [10]:
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
sil = []
kmax = 5

# dissimilarity would not be defined for a single cluster, thus, minimum number of clusters should be 2
for k in range(2, kmax+1):
    kmeans = KMeans(n_clusters = k,random_state=0).fit(hotspot_grouped.drop(["Neighborhood"], axis=1))
    labels = kmeans.labels_
    sil.append(silhouette_score(hotspot_grouped.drop(["Neighborhood"], axis=1), labels, metric = 'euclidean'))
sil

[0.3625293590571335,
 0.36739356156011477,
 0.17105991161376116,
 0.10009019808295974]

In [11]:
kmean = KMeans(init="k-means++", n_clusters=3, random_state=0).fit(hotspot_grouped.drop(["Neighborhood"], axis=1))
cluster = pd.DataFrame({"Cluster":kmean.labels_})

In [12]:
neighbor_list = []
for x in hotspot_rank.Neighborhood:
    neighbor_list.append(x)
latlng = df.set_index ("Neighborhood").loc[neighbor_list, ["Latitude", "Longitude"]].reset_index().drop(["Neighborhood"], axis=1)
vn_merged = pd.concat([cluster, hotspot_rank, latlng], axis=1).set_index("Neighborhood").reset_index()
vn_merged

,Neighborhood,Cluster,The 1st famous,The 2nd famous,The 3rd famous,The 4th famous,The 5th famous,The 6th famous,The 7th famous,The 8th famous,The 9th famous,The 10th famous,The 11th famous,The 12th famous,The 13th famous,The 14th famous,The 15th famous,Latitude,Longitude
0,Binh Tan,0,Flea Market,Hotel,Dance Studio,Argentinian Restaurant,Multiplex,Opera House,North Indian Restaurant,Noodle House,Museum,Mexican Restaurant,Middle Eastern Restaurant,Massage Studio,Market,Lounge,Korean Restaurant,10.765238,106.603853
1,Binh Thanh,1,Vietnamese Restaurant,Coffee Shop,IT Services,Colombian Restaurant,Bookstore,Food Service,Food Truck,Museum,Opera House,North Indian Restaurant,Noodle House,Multiplex,Pizza Place,Middle Eastern Restaurant,Mexican Restaurant,10.810574,106.709134
2,District 1,1,Coffee Shop,Vietnamese Restaurant,Hotel,Massage Studio,French Restaurant,Italian Restaurant,Hotel Bar,Brewery,Hotpot Restaurant,Asian Restaurant,Bar,Japanese Restaurant,Fast Food Restaurant,Sushi Restaurant,Tea Room,10.779320,106.700059
3,District 2,1,Coffee Shop,Vietnamese Restaurant,Restaurant,Argentinian Restaurant,Middle Eastern Restaurant,North Indian Restaurant,Noodle House,Museum,Multiplex,Mexican Restaurant,Park,Massage Studio,Market,Lounge,Korean Restaurant,10.787182,106.749557
4,District 3,1,Vietnamese Restaurant,Coffee Shop,French Restaurant,Seafood Restaurant,Asian Restaurant,BBQ Joint,Bar,Lounge,Breakfast Spot,Noodle House,Rock Club,Juice Bar,Spa,Cupcake Shop,Argentinian Restaurant,10.784265,106.684386
5,District 4,1,Seafood Restaurant,Snack Place,Vietnamese Restaurant,Coffee Shop,Brewery,Pizza Place,Grocery Store,Gym / Fitness Center,Mexican Restaurant,Japanese Restaurant,Asian Restaurant,Korean Restaurant,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Massage Studio,10.757808,106.701283
6,District 7,1,Vietnamese Restaurant,Flea Market,Coffee Shop,BBQ Joint,Bed & Breakfast,Residential Building (Apartment / Condo),Scandinavian Restaurant,Sushi Restaurant,Gym / Fitness Center,Argentinian Restaurant,Museum,North Indian Restaurant,Noodle House,Mexican Restaurant,Multiplex,10.734065,106.721623
7,Phu Nhuan,1,Coffee Shop,Vietnamese Restaurant,Gym / Fitness Center,BBQ Joint,Bookstore,Noodle House,Pizza Place,Multiplex,Opera House,North Indian Restaurant,Museum,Argentinian Restaurant,Middle Eastern Restaurant,Mexican Restaurant,Massage Studio,10.799217,106.680260
8,Tan Binh,1,Vietnamese Restaurant,Coffee Shop,Supermarket,Seafood Restaurant,BBQ Joint,Asian Restaurant,Breakfast Spot,Sushi Restaurant,Bubble Tea Shop,Flower Shop,Multiplex,Gym / Fitness Center,Shopping Plaza,North Indian Restaurant,Noodle House,10.801498,106.652643
9,Thu Duc,2,Pizza Place,Market,Noodle House,Argentinian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Lounge,Massage Studio,Mexican Restaurant,Middle Eastern Restaurant,Multiplex,Museum,North Indian Restaurant,Opera House,10.849422,106.753664


In [13]:
import matplotlib.cm as cm
import matplotlib.colors as colors


# create map
map_clusters = folium.Map(location=[lat, long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(5)
ys = [i + x + (i*x)**2 for i in range(5)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(vn_merged['Latitude'], vn_merged['Longitude'], vn_merged['Neighborhood'], vn_merged['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [14]:
df_1[df_1["Neighborhood"]=="District 3"]["Venue Category"].value_counts().to_frame()

,Venue Category
Vietnamese Restaurant,12
Coffee Shop,9
French Restaurant,3
Seafood Restaurant,3
Asian Restaurant,2
Noodle House,1
Breakfast Spot,1
BBQ Joint,1
Spa,1
Bar,1


In [15]:
df_1[df_1["Neighborhood"]=="District 1"]["Venue Category"].value_counts().to_frame()

,Venue Category
Coffee Shop,18
Vietnamese Restaurant,7
Hotel,6
Massage Studio,4
Italian Restaurant,3
Hotel Bar,3
French Restaurant,3
Brewery,2
Bar,2
Fast Food Restaurant,2


## Conclusion

As you can see the 3 districts that is close to the center are district 1,3,4. In the top 5 famous venues in
district 3, 4 of them are different culture restaurants. It seem like people around this area love restaurant 
and especially different culture restaurants. However, there are no Italian restaurant here yet, hence there are
less competetor. As the result, we should open the restaurant somewhere in district 3 because it is close to the
center of the city, people love different culture restaurant, and there are no Italian restaurant here. A reason for not open it in district 1 is because it is the center of the city so the rent will be expensive, plus there are 
3 Italian restaurants there so it is very competitive.